In [18]:
import pandas as pd
train= pd.read_csv("C:/Users/anujq/OneDrive/Documents/Dissertations/HealthCare_analysis/Dataset/train_data.csv")
test= pd.read_csv("C:/Users/anujq/OneDrive/Documents/Dissertations/HealthCare_analysis/Dataset/test_data.csv")

import numpy as np
df_train = pd.concat([train, test])



In [ ]:
#!pip install keras==2.13.1
#!pip install tensorflow==2.8.0
#!pip install tensorflow_addons

In [4]:
! pip install nltk

     ---------------------------------------- 0.0/41.5 kB ? eta -:--:--
     -------------------------------------- 41.5/41.5 kB 680.6 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.2/1.5 MB 3.5 MB/s eta 0:00:01
   ----------- ---------------------------- 0.5/1.5 MB 5.6 MB/s eta 0:00:01
   ---------------------- ----------------- 0.8/1.5 MB 6.0 MB/s eta 0:00:01
   ------------------------------ --------- 1.2/1.5 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------  1.5/1.5 MB 6.7 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 6.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.7 kB ? eta -:--:--
   ------------------------------------ --- 245.8/269.7 kB 7.6 MB/s eta 0:00:01
   ---------------------------------------- 269.7/269.7 kB 5.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\anujq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\anujq\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\anujq\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [7]:
! pip install textblob

   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
    --------------------------------------- 10.2/626.3 kB ? eta -:--:--
   ----- ---------------------------------- 92.2/626.3 kB 1.3 MB/s eta 0:00:01
   -------------------- ------------------- 327.7/626.3 kB 3.4 MB/s eta 0:00:01
   ------------------------------------ --- 573.4/626.3 kB 3.6 MB/s eta 0:00:01
   ---------------------------------------- 626.3/626.3 kB 3.6 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\anujq\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [39]:
#Importing necessary libraries
import nltk
import pandas as pd
from textblob import Word
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer, one_hot
from keras.utils import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf


#Pre-Processing the text
def cleaning(df, stop_words):
    df_train['Review'] = df_train['FFT answer'].apply(lambda x: ' '.join(x.lower() for x in str(x).split()))
    # Replacing the digits/numbers
    df['Review'] = df['Review'].str.replace('d', '')
    # Removing stop words
    df['Review'] = df['Review'].apply(lambda x: ' '.join(x for x in x.split() if x not in stop_words))
    # Lemmatization
    df['Review'] = df['Review'].apply(lambda x: ' '.join([Word(x).lemmatize() for x in x.split()]))
    return df

stop_words = stopwords.words('english')
data_cleaned = cleaning(df_train, stop_words)

#Generating Embeddings using tokenizer
tokenizer = Tokenizer(num_words=100, split=' ')
tokenizer.fit_on_texts(data_cleaned['Review'].values)
X = tokenizer.texts_to_sequences(data_cleaned['Review'].values)
X = pad_sequences(X)

#X= tf.keras.utils.normalize(X, axis=-1, order=2)

# one hot encoding on target
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()
Y= df_train["Comment sentiment"].values
Y= enc.fit_transform(Y.reshape(-1,1)).toarray()

# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.25, random_state=5)


In [33]:
X

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 3],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 2],
       [0, 0, 0, ..., 0, 0, 2]])

In [40]:
Y

array([[0., 0., 0., 0., 0., 1.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.]])

In [42]:
#Model Building
import keras
from keras import optimizers
model = Sequential()
model.add(Embedding(100, 160, input_length = X.shape[1]))
model.add(Dropout(0.2))
model.add(SpatialDropout1D(0.3))
model.add(LSTM(512, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(256, activation='LeakyReLU'))
model.add(Dense(128, activation='LeakyReLU'))
model.add(Dense(64, activation='LeakyReLU'))
model.add(Dense(6, activation='softmax'))

adam = optimizers.Adam(lr=0.05)
model.compile(loss = 'categorical_crossentropy', optimizer=adam, metrics = ['accuracy',keras.metrics.F1Score()])
print(model.summary())

#Model Training
history=model.fit(X_train, Y_train, epochs = 50, batch_size=32, verbose =1,validation_split=0.1)

#Model Testing
score, acc, f1_score = model.evaluate(X_test,Y_test)



print("loss:",score)
print("accuracy",acc)
print("f1_score",f1_score)








Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_11 (Embedding)    (None, 83, 160)           16000     
                                                                 
 dropout_4 (Dropout)         (None, 83, 160)           0         
                                                                 
 spatial_dropout1d_4 (Spati  (None, 83, 160)           0         
 alDropout1D)                                                    
                                                                 
 lstm_11 (LSTM)              (None, 512)               1378304   
                                                                 
 dense_36 (Dense)            (None, 256)               131328    
                                                                 
 dense_37 (Dense)            (None, 128)               32896     
                                                     

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
#model.save('model.h5')